In [2]:
import ee 
import geemap
from geemap.datasets import DATA
from geemap import colormaps
import os
import pandas as pd

In [3]:
ee.Initialize()

In [4]:
# change bands for Landsat in question
def getNDVI(img):
  return img.addBands(img.normalizedDifference(['B5','B4']).rename('NDVI'))

In [6]:
geometry = ee.FeatureCollection('users/Samuelvalman/site')
uk = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filter(ee.Filter.eq("country_co", "UK"))

ic = ee.ImageCollection("LANDSAT/LC08/C01/T1_TOA").filterBounds(geometry).map(lambda image:image.clip(geometry))

# number of all collected images
nbr = ic.size().getInfo()
print(nbr)

5215


In [5]:
Map = geemap.Map()
Map.addLayer(ee.Image(loi.get(4)), {}, 'x')
Map.centerObject(geometry, zoom =15)
Map

NameError: name 'loi' is not defined

In [6]:
ic = ic.map(getNDVI)

In [7]:
# landsat simple cloud score for each pixel 
def cloud_func(image):
    return ee.Algorithms.Landsat.simpleCloudScore(image)
ic = ic.map(cloud_func)

In [8]:
Map = geemap.Map()


In [9]:
loi = ic.toList(ic.size())
#size = 539
region = geometry.geometry().bounds().getInfo()
out_dir = os.path.join(os.path.expanduser('~'), 'Downloads')

In [10]:
def mean_cloud(image):
    cloud = image.select('cloud')
    mean = cloud.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': region,
    'scale': 30
    })
    return mean

In [11]:
def NDVI_mean(image):
    cloud = image.select('NDVI')
    mean = cloud.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': region,
    'scale': 30
    })
    return mean

In [12]:
def B3_mean(image):
    cloud = image.select('B3')
    mean = cloud.reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': region,
    'scale': 30
    })
    return mean

In [14]:
date_list = []
ND = []
B3 = []
ticker = 0 


for i in range(0,222):
    ticker +=1
    print(ticker)
    im = ee.Image(loi.get(i))
    meancloud = mean_cloud(im)
    meancloud = meancloud.getInfo()
    meancloud = meancloud['cloud']
    try:
        meancloud = float(meancloud)
    except TypeError:
        meancloud = float(0)
    
    if meancloud <= 0.1:
        meanNDVI = NDVI_mean(im)
        meanNDVI = meanNDVI.getInfo()
        meanNDVI = meanNDVI['NDVI']
        ND.append(meanNDVI)
        
        m3 = B3_mean(im)
        m3 = m3.getInfo()
        m3 = m3['B3']
        B3.append(meanNDVI)
        
        
        
        date = im.get('system:time_start')
        date = ee.Date(date)
        date = date.format("YYYY-MM-dd")
        date = str(date.getInfo())
        date_list.append(date)

ls = pd.DataFrame({'date':date_list, 'NDVI':ND, 'b3':B3})

        


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222


In [19]:
# ls.to_csv('ls8.csv')